
# Dependencies

In [24]:
import custom_models
#python packages
from PIL import Image
from tqdm.notebook import tqdm
#from tqdm import tqdm
import gc
import datetime
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from skimage import io
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
#torchvision
import torchvision
from torchvision import datasets, models, transforms
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version:  1.2.0
Torchvision Version:  0.4.0a0+6b959ee
Using GPU!


# Initialize CNN model

In [25]:
def make_CNN(model_name, num_classes, resume_from = None):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    input_size = 0
    
    # You may NOT use pretrained models!! 
    use_pretrained = False
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg13":
        """ VGG13_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg16":
        """ VGG16_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "vgg19":
        """ VGG19_bn
        """
        model_ft = custom_models.vgg13_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    else:
        raise Exception("Invalid model name!")
    
    return model_ft, input_size

# Define Multimodal Model

In [26]:
class Chest_Disease_Net(nn.Module):
    """
    fc1: number of neurons in the hidden fully connected layer
    """
    def __init__(self, cnn_model_name, num_classes, num_multimodal_features=12, fc1_out=32, resume_from=None):
        #num_classes = 14
        #num_multimodal_features= 12
        super(Chest_Disease_Net, self).__init__()
        self.cnn, self.input_size = make_CNN(cnn_model_name, num_classes)#models.vgg11(pretrained=False, progress = True)
        #define output layers
        self.fc1 = nn.Linear(num_classes + num_multimodal_features, fc1_out) #takes in input of CNN and multimodal input
        self.fc2 = nn.Linear(fc1_out, num_classes)
        if resume_from is not None:
            print("Loading weights from %s" % resume_from)
            self.load_state_dict(torch.load(resume_from))
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        #print("x1", x1.shape)
        x2 = data
        #print("x2", x2.shape)
        #print("x1: ", x1, type(x1))
        #print("x2: ", x2, type(x2))
        #x = torch.cat((x1, x2), dim=1)  
        x = torch.cat((x1.float(), x2.float()), dim=1) ### ???
        #print("concat", x.shape)
        x = F.relu(self.fc1(x))
        #print("relu", x.shape)
        x = self.fc2(x)
        print('forward output: ', x)
       # print("fc2", x.shape)
        return x.double() ### ???

# Data Loading

In [27]:
class MultimodalDataset(Dataset):
    """
    Custom dataset definition
    """
    def __init__(self, csv_path, img_path, transform=None):
        """
        """
        self.df = pd.read_csv(csv_path)
        self.img_path = img_path
        self.transform = transform
        self.diseases = self.get_diseases()
        
            
    def __getitem__(self, index):
        """
        """
        img_name = self.df.iloc[index]["img_name"] 
        img_path = os.path.join(self.img_path, img_name)
        image = Image.open(img_path)
        image = image.convert("RGB")
        image = np.asarray(image)
        if self.transform is not None:
            image = self.transform(image)
        dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor # ???
        features = np.fromstring(self.df.iloc[index]["feature"][1:-1], sep=",") # ???
        features = torch.from_numpy(features.astype("float")) # ???
        #label = int(self.df.iloc[index]['label'])
        labels = torch.tensor(list(self.df.iloc[index][self.diseases]), dtype = torch.float64)
        #print("Label type: ", type(label))
        #label = np.int_(label) #???
        #print("label type post casting: ", type(label))
        return image, features, labels
        
    def __len__(self):
        return len(self.df)
    
    def get_diseases(self):
        cols = list(self.df.columns)
        cols.remove('disease')
        cols.remove('feature')
        cols.remove('img_name')
        cols.remove('dataset_type')
        return cols
        

In [28]:
def get_dataloaders(input_size, batch_size, num_classes, augment=False, shuffle = True):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            #Add extra transformations for data augmentation
            transforms.RandomApply([
                transforms.RandomChoice([
                    transforms.RandomAffine(degrees=20),
                    transforms.RandomAffine(degrees=0,scale=(0.1, 0.15)),
                    transforms.RandomAffine(degrees=0,translate=(0.2,0.2)),
                    #transforms.RandomAffine(degrees=0,shear=0.15),
                    transforms.RandomHorizontalFlip(p=1.0)
                ] if augment else [transforms.RandomAffine(degrees=0)])#else do nothing
            ], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'test': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    # Create training and validation datasets
    data_subsets = {x: MultimodalDataset(csv_path="./data/"+x+"_dataset{}.csv".format(num_classes), 
                                         img_path="/storage/images", 
                                         transform=data_transforms[x]) for x in data_transforms.keys()}
    # Create training and validation dataloaders
    # Never shuffle the test set
    dataloaders_dict = {x: DataLoader(data_subsets[x], batch_size=batch_size, shuffle=False if x != 'train' else shuffle, num_workers=4) for x in data_transforms.keys()}
    return dataloaders_dict

# Training

In [36]:
def train_model(model, multimodal, dataloaders, criterion, optimizer, scheduler, model_name=str(datetime.datetime.now()), 
                save_dir = None, save_all_epochs=False, num_epochs=25):
    '''
    model: The NN to train
    dataloaders: A dictionary containing at least the keys 
                 'train','val' that maps to Pytorch data loaders for the dataset
    criterion: The Loss function
    optimizer: The algorithm to update weights 
               (Variations on gradient descent)
    num_epochs: How many epochs to train for
    save_dir: Where to save the best model weights that are found, 
              as they are found. Will save to save_dir/weights_best.pt
              Using None will not write anything to disk
    save_all_epochs: Whether to save weights for ALL epochs, not just the best
                     validation error epoch. Will save to save_dir/weights_e{#}.pt
    '''
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # TQDM has nice progress bars
            for inputs, features, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                features = features.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    if multimodal:
                        outputs = model(inputs, features)
                    else:
                        outputs = model(inputs).type(torch.float64)
                    print("model outputs: ", outputs, outputs.size())
                    #print("model labels: ", labels.size())
                    loss = criterion(outputs, labels)

                    # torch.max outputs the maximum value, and its index
                    # Since the input is batched, we take the max along axis 1
                    # (the meaningful outputs)
                    #print("outputs: ", outputs)
                    #preds = torch.max(outputs, 1)
                    preds = (outputs > 0).type(torch.float64)
                    #_, preds = torch.max(outputs, 1)
                    #print("new preds: ", preds)
                    # backprop + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                #print("loss: ", loss.item())
                #print("inputs: ", inputs.size(0), inputs.size())
                
                running_loss += loss.item() * inputs.size(0)
                #print("running loss: ", running_loss)
                print("predictions: ", preds, preds.size())
                print("Labels: ", labels.data, labels.data.size())
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            #is the accuracy calculated correctly?
            print("running_corrects: ", running_corrects.double(), running_corrects.size())
            print("dataloaders len: ", len(dataloaders[phase].dataset)) 
            #maybe dataloaders length * number of classes? Model must predict for all classes
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), save_dir + "/{}_best_weights_1.pt".format(model_name))
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print()
        scheduler.step()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

# Optimizer & Loss

In [30]:
def make_optimizer(model):
    # Get all the parameters
    params_to_update = model.parameters()
    print("Params to learn:")
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

    # Use SGD
    optimizer = optim.SGD(params_to_update, lr=0.01, momentum=0.9)
    return optimizer

def get_loss(num_classes,device):
    # Create an instance of the loss function
    #set weights to account for unbalanced data.
    #In expectation every category class contributes the same to the loss
    pos_weight = np.array([    0.025944895136267, 0.059170436277992,0.049543908183484,
                            0.124453250588807,0.013364985606939,0.185904145949381,
                            0.108022729821676,0.564067815619275,0.054011364910838])
    pos_weight = torch.tensor(pos_weight,dtype=torch.float64) if num_classes == 9 \
    else torch.tensor([])
    pos_weight = pos_weight.to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    return criterion

# Define Experiment Parameters

In [34]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
# You can add your own, or modify these however you wish!
model_name = "alexnet"

# Number of classes in the dataset
# Miniplaces has 100
num_classes = 9# set to 9 or 14

# Batch size for training (change depending on how much memory you have)
# You should use a power of 2.
batch_size = 64

# Shuffle the input data?
shuffle_datasets = True

# Number of epochs to train for 
num_epochs = 10

### IO
# Path to a model file to use to start weights at
#resume_from = "/home/ubuntu/6.867-xray-project/weights/data_aug_vgg.pt"
resume_from = None

# Directory to save weights to
save_dir = "weights"
os.makedirs(save_dir, exist_ok=True)

# If True saves the weights for all epochs, else only saves the weight of best one
save_all_epochs = False

#Enable multimodal
multimodal = False

# Train!

In [32]:
gc.collect()

990

In [37]:
# Initialize the model for this run
if multimodal:
    model = Chest_Disease_Net(cnn_model_name = model_name, num_classes = num_classes, resume_from = resume_from)
    input_size = model.input_size
else:
    model, input_size = make_CNN(model_name=model_name,num_classes=num_classes, resume_from = resume_from)
    
dataloaders = get_dataloaders(input_size, batch_size, num_classes, shuffle_datasets)
criterion = get_loss(num_classes=num_classes,device=device)

# Move the model to the gpu if needed
model = model.to(device)

optimizer = make_optimizer(model)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,10],gamma=0.1)
# Train the model!
trained_model, validation_history = train_model(model=model, multimodal=multimodal,
                                                dataloaders=dataloaders, criterion=criterion, optimizer=optimizer,
            scheduler=scheduler, model_name=model_name, save_dir=save_dir, save_all_epochs=save_all_epochs, num_epochs=num_epochs)

Params to learn:
	 features.0.weight
	 features.0.bias
	 features.3.weight
	 features.3.bias
	 features.6.weight
	 features.6.bias
	 features.8.weight
	 features.8.bias
	 features.10.weight
	 features.10.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias
Epoch 0/9
----------


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f38010fe828>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f38010fe828>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.d

model outputs:  tensor([[-2.0565e-02,  2.6612e-03, -3.3670e-03, -1.2585e-02,  2.5765e-02,
         -6.6867e-03, -1.0234e-02,  2.2622e-02, -2.6087e-03],
        [-1.3005e-02,  4.3028e-03, -3.9609e-03,  2.9693e-03,  1.0114e-02,
         -6.2731e-03, -3.4548e-02,  1.7021e-02, -1.1387e-04],
        [-1.7200e-02,  1.2711e-03, -3.2782e-03, -1.2602e-02,  2.1503e-02,
         -4.7971e-03, -1.1838e-02,  1.1922e-02,  7.9263e-04],
        [-1.9981e-02, -2.2260e-04,  5.3618e-03, -8.8174e-03,  1.2821e-02,
         -9.2966e-03, -1.3570e-02,  1.8350e-02, -4.6068e-03],
        [-1.2018e-02,  1.7674e-02, -1.5926e-03, -3.6958e-03,  2.0728e-02,
         -2.3673e-03, -3.0524e-03,  1.3564e-02,  2.0132e-03],
        [-6.4156e-03,  2.4536e-03,  1.1266e-02, -1.4409e-02,  1.7443e-02,
         -1.2461e-02, -1.0097e-02,  1.0281e-02,  4.1139e-03],
        [-1.3825e-02,  4.9947e-03, -2.5706e-03, -9.6975e-03,  1.7101e-02,
         -1.1029e-02, -8.3604e-03,  1.5776e-02,  4.8307e-03],
        [-2.0158e-02,  1.7359e-0

model outputs:  tensor([[-2.1693e-02,  1.1289e-03, -2.6041e-04, -2.2622e-02,  1.0383e-02,
         -6.9136e-03, -8.3866e-03,  1.4565e-02, -5.1682e-03],
        [-1.7584e-02,  2.7874e-03,  4.0967e-03, -1.2023e-02,  1.2799e-02,
          1.4302e-03, -1.2700e-02,  1.7316e-02, -2.1172e-03],
        [-1.0435e-02, -1.5543e-03,  6.5548e-04, -1.0702e-02,  1.2984e-02,
         -1.5426e-02, -1.0116e-02,  2.0192e-02, -5.7376e-03],
        [-1.8736e-02, -2.1874e-03, -3.7227e-05, -1.9706e-02,  1.7119e-02,
         -6.5886e-03, -9.1857e-03,  1.3764e-02, -7.6683e-03],
        [-2.3306e-02, -3.0764e-04, -7.6023e-03, -1.0897e-02,  1.1672e-02,
         -8.7115e-03, -1.6854e-02,  1.0878e-02, -1.7788e-03],
        [-2.2191e-02,  9.1034e-03,  1.6501e-06, -1.9022e-02,  8.1750e-03,
         -6.4274e-03, -1.7283e-02,  1.8348e-02,  5.2670e-03],
        [-1.8590e-02, -6.8796e-04, -1.0882e-03, -1.4048e-02,  1.4789e-02,
         -4.4947e-03, -8.9878e-03,  2.2418e-02,  1.9304e-03],
        [-1.9534e-02,  1.2054e-0

model outputs:  tensor([[-0.0210, -0.0111,  0.0014, -0.0149,  0.0055, -0.0141, -0.0198,  0.0185,
         -0.0022],
        [-0.0257, -0.0077, -0.0093, -0.0154,  0.0048, -0.0184, -0.0188,  0.0149,
         -0.0025],
        [-0.0211, -0.0027, -0.0122, -0.0157,  0.0123, -0.0166, -0.0242,  0.0208,
         -0.0059],
        [-0.0213, -0.0072, -0.0095, -0.0207,  0.0085, -0.0126, -0.0187,  0.0184,
         -0.0060],
        [-0.0229, -0.0012, -0.0107, -0.0246,  0.0112, -0.0116, -0.0181,  0.0154,
         -0.0060],
        [-0.0206, -0.0013, -0.0039, -0.0080,  0.0038, -0.0144, -0.0186,  0.0137,
         -0.0072],
        [-0.0223, -0.0015, -0.0064, -0.0190,  0.0040, -0.0168, -0.0151,  0.0084,
         -0.0100],
        [-0.0238, -0.0046,  0.0009, -0.0227,  0.0125, -0.0079, -0.0119,  0.0104,
         -0.0046],
        [-0.0307, -0.0151, -0.0176, -0.0217,  0.0172, -0.0152, -0.0153,  0.0193,
         -0.0183],
        [-0.0219, -0.0116, -0.0024, -0.0253,  0.0141, -0.0155, -0.0220,  0.0101,
   

...

model outputs:  tensor([[-8.5594, -7.1908, -7.7534, -6.5461, -9.3975, -4.2554, -6.0883, -1.3341,
         -8.0567],
        [-3.6479, -3.1378, -3.1310, -2.5830, -3.8771, -1.7809, -2.8661, -0.4073,
         -3.2145],
        [-7.1669, -5.7278, -6.3556, -5.1716, -7.3749, -3.3241, -5.0320, -0.7976,
         -6.2566],
        [-6.7792, -5.9609, -6.1814, -5.1032, -7.3971, -3.4311, -4.9713, -0.9043,
         -6.2694],
        [-6.8440, -5.6445, -6.0179, -4.6640, -6.8798, -3.1091, -5.0216, -0.5903,
         -5.9224],
        [-7.6198, -6.5629, -6.7288, -5.3145, -8.1243, -3.4685, -5.5051, -1.1455,
         -6.8456],
        [-5.3386, -4.5094, -4.9783, -3.9729, -5.7727, -2.6014, -4.0067, -0.6513,
         -4.7747],
        [-4.6721, -3.8522, -4.0386, -3.2770, -4.9621, -2.3667, -3.1979, -0.4124,
         -3.9383],
        [-6.9884, -5.6932, -6.0413, -4.8184, -7.4058, -3.2488, -4.8790, -0.5801,
         -6.2873],
        [-6.4263, -5.2447, -5.5666, -4.5503, -6.9681, -2.9978, -4.8456, -0.7303,
   

model outputs:  tensor([[-8.5012, -7.2094, -7.5837, -6.2840, -9.1823, -4.1680, -6.2796, -0.8625,
         -7.8275],
        [-9.2406, -7.2769, -7.8477, -6.2179, -9.2811, -4.2848, -6.5141, -0.3318,
         -8.0401],
        [-6.9011, -5.9209, -6.4849, -5.3911, -7.4712, -3.4906, -5.1977, -0.4666,
         -6.3778],
        [-4.8934, -3.9023, -4.4349, -3.3860, -5.0688, -2.2763, -3.4054, -0.5344,
         -4.1830],
        [-6.4440, -5.6969, -6.1031, -4.7448, -7.2551, -3.1360, -5.0738, -0.5617,
         -5.8572],
        [-6.8149, -5.8873, -6.2617, -5.1470, -7.0152, -3.5092, -4.9844, -0.4470,
         -6.1779],
        [-9.1553, -8.1956, -8.4118, -6.6611, -9.8031, -4.3184, -6.8232, -1.1028,
         -8.1874],
        [-7.3739, -6.0613, -6.2994, -5.2104, -7.9062, -3.5478, -5.4838, -0.3096,
         -6.5126],
        [-6.8042, -5.4886, -5.8227, -4.9437, -6.7096, -3.1714, -4.8264, -0.4413,
         -6.0955],
        [-6.6672, -5.8205, -6.1085, -4.9125, -7.2563, -3.2127, -5.0548, -0.8736,
   

model outputs:  tensor([[-8.1500e+00, -6.7637e+00, -6.8964e+00, -5.5557e+00, -8.5758e+00,
         -3.8296e+00, -5.9724e+00,  1.5305e-01, -7.1723e+00],
        [-7.0623e+00, -6.0700e+00, -6.2599e+00, -4.7569e+00, -7.4847e+00,
         -3.4216e+00, -5.2118e+00, -6.6689e-02, -6.6185e+00],
        [-8.0966e+00, -6.7706e+00, -7.1901e+00, -5.4747e+00, -8.4707e+00,
         -3.7765e+00, -5.9904e+00, -5.6272e-02, -6.8102e+00],
        [-8.0149e+00, -6.7240e+00, -7.2663e+00, -5.6552e+00, -8.0184e+00,
         -3.6307e+00, -5.7746e+00,  3.3686e-01, -7.0218e+00],
        [-7.1381e+00, -6.1560e+00, -6.4054e+00, -5.3211e+00, -7.6259e+00,
         -3.4613e+00, -5.4655e+00,  4.7187e-01, -6.3144e+00],
        [-5.2389e+00, -4.6250e+00, -4.8569e+00, -3.8518e+00, -5.8486e+00,
         -2.6046e+00, -3.8331e+00,  4.0311e-02, -4.5643e+00],
        [-6.6080e+00, -5.5342e+00, -5.5942e+00, -4.8105e+00, -6.9082e+00,
         -3.1980e+00, -4.5710e+00,  3.5018e-02, -5.7424e+00],
        [-7.3171e+00, -6.0696e+0

model outputs:  tensor([[-7.3472e+00, -6.3260e+00, -6.7156e+00, -4.9594e+00, -7.7594e+00,
         -3.8280e+00, -5.5411e+00,  3.2482e-04, -6.6613e+00],
        [-4.7271e+00, -4.1291e+00, -4.1852e+00, -3.3214e+00, -5.0275e+00,
         -2.3293e+00, -3.3900e+00,  6.2115e-02, -4.1536e+00],
        [-8.8253e+00, -7.3814e+00, -7.4198e+00, -6.0832e+00, -9.5357e+00,
         -4.1452e+00, -6.5995e+00,  1.6988e-01, -7.7739e+00],
        [-6.3872e+00, -5.4115e+00, -5.6925e+00, -4.4367e+00, -6.6215e+00,
         -3.0966e+00, -4.7133e+00,  6.0477e-02, -5.5109e+00],
        [-7.5629e+00, -6.9591e+00, -7.1075e+00, -5.5267e+00, -8.6438e+00,
         -3.7400e+00, -5.5866e+00,  2.9674e-01, -6.9998e+00],
        [-3.9693e+00, -3.2794e+00, -3.4360e+00, -2.6434e+00, -4.0887e+00,
         -1.9886e+00, -2.7686e+00,  2.1298e-01, -3.4026e+00],
        [-7.1650e+00, -6.0837e+00, -6.1376e+00, -4.9473e+00, -7.4043e+00,
         -3.4603e+00, -5.1248e+00,  3.8875e-01, -6.3417e+00],
        [-7.9717e+00, -6.9432e+0

KeyboardInterrupt: 